In [1]:
import sys
import os
import glob # 用于查找文件
from pathlib import Path
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
import numpy as np
import pandas as pd
from pycaret.regression import load_model, predict_model
from src.config.config import Config
from datetime import datetime

def predict_future_scenario(data: pd.DataFrame, model_path: str):
    """使用加载的模型对单个场景的未来特征数据进行预测

    Args:
        data: 单个场景的、经过特征工程处理的DataFrame
        model_path: 已训练模型的路径 (不含扩展名)

    Returns:
        pd.DataFrame: 包含原始数据和预测结果 ('prediction_label' 和 'MSW_pred') 的DataFrame
    """
    # 加载模型
    model = load_model(model_path)
    
    # 预测 (PyCaret 会添加 'prediction_label' 列)
    predictions_df = predict_model(model, data=data)
    
    # 将预测结果转换回原始尺度 (假设目标变量进行了 log1p 转换)
    # !! 如果您的目标变量转换方式不同，请修改这里的逆转换 !!
    target_column_pred = f"{Config.DATA_CONFIG['target_column']}_pred"
    predictions_df[target_column_pred] = np.expm1(predictions_df['prediction_label'])
    
    return predictions_df


In [2]:
# --- 主程序 ---

# 1. 定义模型路径 (与 predict_ext.ipynb 中使用的模型相同)
# !! 请确保这里的模型名称是您想要使用的最终模型 !!
model_name = 'gbr_250412_192557' # 示例模型名称，请替换为您实际的模型
model_path = os.path.join(Config.PATH_CONFIG['models_dir'], model_name) 

# 2. 查找所有按场景生成的特征文件
features_dir = Path(Config.PATH_CONFIG['features_dir'])
scenario_feature_files = glob.glob(str(features_dir / 'future_features_*.csv'))

if not scenario_feature_files:
    print(f"错误：在目录 {features_dir} 中未找到任何 'future_features_*.csv' 文件。")
    print("请先运行 data_preprocess.ipynb 生成特征文件。")
    # sys.exit(1) # 在脚本中可以取消注释以退出

# 3. 逐个场景进行预测并收集结果
all_scenario_predictions = []

print(f"开始使用模型 '{model_name}' 对未来场景进行预测...")
for file_path in scenario_feature_files:
    scenario_name = Path(file_path).stem.replace('future_features_', '') # 从文件名提取场景名
    print(f"  正在加载和预测场景: {scenario_name} (文件: {file_path})")
    
    try:
        # 加载特征数据
        scenario_features_df = pd.read_csv(file_path)
        
        # 进行预测
        scenario_predictions_df = predict_future_scenario(scenario_features_df, model_path)
        
        # 添加场景标识列 (如果原始特征文件中没有Scenario列，需要添加回去)
        if 'Scenario' not in scenario_predictions_df.columns:
             scenario_predictions_df['Scenario'] = scenario_name

        all_scenario_predictions.append(scenario_predictions_df)
        print(f"    场景 '{scenario_name}' 预测完成.")
        
    except Exception as e:
        print(f"    处理文件 {file_path} 时出错: {e}")

# 4. 合并所有场景的预测结果
if all_scenario_predictions:
    final_predictions_df = pd.concat(all_scenario_predictions, ignore_index=True)
    
    # 5. 保存合并后的预测结果
    prediction_dir = Path(Config.PATH_CONFIG['prediction_dir'])
    prediction_dir.mkdir(parents=True, exist_ok=True) # 确保目录存在
    
    timestamp = datetime.now().strftime('%d%m%y_%H%M%S')
    output_file = prediction_dir / f'future_msw_predictions_all_scenarios_{timestamp}.csv'
    
    final_predictions_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n所有场景的合并预测结果已保存至: {output_file}")
else:
    print("\n没有成功生成任何场景的预测结果。")

开始使用模型 'gbr_250412_192557' 对未来场景进行预测...
  正在加载和预测场景: SSP1 (文件: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\future_features_SSP1.csv)
Transformation Pipeline and Model Successfully Loaded
    场景 'SSP1' 预测完成.
  正在加载和预测场景: SSP2 (文件: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\future_features_SSP2.csv)
Transformation Pipeline and Model Successfully Loaded
    场景 'SSP2' 预测完成.
  正在加载和预测场景: SSP3 (文件: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\future_features_SSP3.csv)
Transformation Pipeline and Model Successfully Loaded
    场景 'SSP3' 预测完成.
  正在加载和预测场景: SSP4 (文件: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\future_features_SSP4.csv)
Transformation Pipeline and Model Successfully Loaded
    场景 'SSP4' 预测完成.
  正在加载和预测场景: SSP5 (文件: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\future_features_SSP5.csv)
Transformation Pipeline and Model Successfully Loaded
    场景 'SSP5' 预测完成.

所有场景的合并预测结果已保存至: e:\code\jupy

In [3]:
# 6. !! 新增：筛选指定列并保存简化版预测结果 !!
simplified_columns = [
    'Year', 
    'Country Name', 
    'Population', 
    'GDP PPP 2017', 
    'GDP PPP/capita 2017', 
    'Urban population %',
    'Region', 
    'Income Group', 
    'Scenario',
    f"{Config.DATA_CONFIG['target_column']}_pred" # 使用配置中的目标列名加上_pred后缀
]

# 检查所有需要的列是否存在于DataFrame中
missing_cols = [col for col in simplified_columns if col not in final_predictions_df.columns]
if missing_cols:
    print(f"\n警告：在最终结果中找不到以下列，无法生成简化文件: {missing_cols}")
else:
    simplified_df = final_predictions_df[simplified_columns]
    
    # 定义简化版文件的输出路径
    output_file_simplified = prediction_dir / f'future_msw_predictions_simplified_{timestamp}.csv'
    
    # 保存简化版文件
    simplified_df.to_csv(output_file_simplified, index=False, encoding='utf-8-sig')
    print(f"简化版预测结果 (仅含指定列) 已保存至: {output_file_simplified}")

简化版预测结果 (仅含指定列) 已保存至: e:\code\jupyter\固废产生\SW-Prediction\iw\src\results\prediction\future_msw_predictions_simplified_210425_013134.csv
